# Приложение для книголюбов

В "эпоху" короновируса многие люди нашли досуг в чтении книг. Ввиду этого мобильные приложения для тех, кто любит читать, очень актуальны. Чтобы сформулировать ценностное предложение для нового продукта, необходимо проанализировать данные конкурентов и сделать выводы.

На входе есть пять таблиц с данными о книгах, авторах, издательствах, пользовательских рейтингах и обзорах книг. Все таблицы имеют первичные ключи, по которым имеют связи друг с другом.

Для ответов на актуальные вопросы создателей приложения необходимо подключиться к БД с данными. Для этого испортируем библиотеки sqlalchemy и pandas, устанавливаем параметры подключения к БД, формируем строку соединения с БД и, наконец, подключаемся.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры подключения к БД
db_config = {'user': 'praktikum_student',             # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp',                             # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', # адрес сервера
 'port': 6432,                                        # порт подключения
 'db': 'data-analyst-final-project-db'}               # название базы данных

# формируем строку соединения с БД
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# поключаемся к БД
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Для начала хочется посмотреть на данные таблиц. Выведем первые пять строк каждой таблицы.

In [2]:
# уберем перенос строки
pd.options.display.expand_frame_repr = False

for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = ''' SELECT *
            FROM ''' + str(table)
    data = pd.io.sql.read_sql(query, con = engine)
    display('Таблица ' + str(table).upper())
    display(data.head())
    display(data.shape)
    display()

'Таблица BOOKS'

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


(1000, 6)

'Таблица AUTHORS'

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


...

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


(6456, 4)

'Таблица REVIEWS'

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


(2793, 4)

Имея в виду связи между таблицами, ответим на следущие вопросы:
    1. Сколько книг вышло после 1 января 2000 года;
    2. Количество обзоров для каждой книги и средняя оценка;
    3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;
    4. Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками;
    5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
    
Чтобы получить ответы на данные вопросы, напишем по каждому запрос (query1, query2 и т.д.)

In [3]:
# Сколько книг вышло после 1 января 2000 года
query1 = ''' SELECT COUNT(b.title) AS count_of_book 
             FROM books b
             INNER JOIN authors a ON b.author_id = a.author_id
             WHERE publication_date > '2000-01-01'
         '''
display(pd.io.sql.read_sql(query1, con = engine))

,count_of_book
0,819


In [4]:
# Количество обзоров для каждой книги и средняя оценка
query2 = ''' WITH book_prev AS (
                                SELECT b.book_id, b.title, avg(r.rating) AS book_rate
                                FROM books b
                                LEFT JOIN ratings r ON b.book_id = r.book_id
                                GROUP BY b.book_id, b.title
                                )
             SELECT b.title, 
                    COUNT(rw.review_id) AS count_of_reviews,
                    b.book_rate AS avg_rate
             FROM book_prev b
             LEFT JOIN reviews rw ON b.book_id = rw.book_id
             GROUP BY b.title, b.book_rate
             ORDER BY 2 DESC
         '''
display(pd.io.sql.read_sql(query2, con = engine))

,title,count_of_reviews,avg_rate
0,Twilight (Twilight #1),7,3.662500
1,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,The Curious Incident of the Dog in the Night-Time,6,4.081081
3,The Road,6,3.772727
4,The Giver (The Giver #1),6,3.750000
...,...,...,...
995,Essential Tales and Poems,0,4.000000
996,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
997,Leonardo's Notebooks,0,4.000000
998,The Natural Way to Draw,0,3.000000


<div class="alert alert-block alert-info">
Использовала подзапрос, который показывает средную оценку книги, а потом еще посчитала кол-во обзоров.
</div>

In [5]:
# Издательство, которое выпустило наибольшее число книг толще 50 страниц
query3 = ''' SELECT p.publisher AS most_popular_publisher,
                    COUNT(b.book_id) AS count_of_books
             FROM publishers p
             INNER JOIN books b ON b.publisher_id = p.publisher_id
             WHERE b.num_pages > 50
             GROUP BY p.publisher
             ORDER BY COUNT(*) DESC
             LIMIT 1
         '''
display(pd.io.sql.read_sql(query3, con = engine))

,most_popular_publisher,count_of_books
0,Penguin Books,42


<div class="alert alert-block alert-info">
Вывела кол-во изданных книг
</div>

In [6]:
# Автор с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками
query4 = ''' SELECT a.author,
                    ROUND(AVG(r.rating), 5) AS highest_avg_rate
             FROM (SELECT r.book_id
                   FROM ratings r
                   GROUP BY r.book_id
                   HAVING count(r.rating_id) >= 50) AS all_books -- все книги с 50 и более оценками
             LEFT JOIN books b   ON b.book_id = all_books.book_id
             LEFT JOIN authors a ON b.author_id = a.author_id
             LEFT JOIN ratings r ON b.book_id = r.book_id
             GROUP BY a.author
             ORDER by 2 DESC
             LIMIT 10
         '''
display(pd.io.sql.read_sql(query4, con = engine))

,author,highest_avg_rate
0,J.K. Rowling/Mary GrandPré,4.28710
1,Markus Zusak/Cao Xuân Việt Khương,4.26415
2,J.R.R. Tolkien,4.24691
3,Louisa May Alcott,4.19231
4,Rick Riordan,4.08065
5,William Golding,3.90141
6,J.D. Salinger,3.82558
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.78947
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.78788
9,Lois Lowry,3.75000


In [7]:
# Среднее количество обзоров от пользователей, которые поставили больше 50 оценок
query5 = ''' WITH prev AS (SELECT username, COUNT(review_id) AS cnt
                           FROM  reviews
                           GROUP BY username)
             SELECT ROUND(AVG(r.cnt)) AS avg_review
             FROM (SELECT r.username
                   FROM ratings r
                   GROUP BY r.username
                   HAVING COUNT(r.rating_id) > 50) AS all_users --все пользователи, что ставили больше 50 оценок
             LEFT JOIN prev r ON r.username = all_users.username    
         '''
display(pd.io.sql.read_sql(query5, con = engine))

,avg_review
0,24.0


<div class="alert alert-block alert-info">
Посчитала среднее по этим пользователям
</div>

**Итого**:
1. После 1 января 2000 года вышло 819 книг;
2. Самые "обсуждаемые" книги с наибольшим количеством обзоров - "Сумерки" (xo-xo). 
3. Издательство, которое выпустило больше всего книг (а не брошюр) - "Penguin Books" - 42 книги
4. Автор с самой высокой средней оценкой книг (4,29) - J.K. Rowling/Mary GrandPré. Что логично - кто не читал книги о Гарри Поттере? 
5. В среднем те, кто поставили более 50 оценок книгам, написали в српеднем 24 обзора.